#### Perform AutoML Job

In [3]:
# Import Libraries 
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential 
from azure.ai.ml import MLClient, Input, automl
from azure.ai.ml.constants import AssetTypes

In [2]:
# get connection to workspace 
try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()
    
# get token 
ws = MLClient.from_config(credential=credential)
print(ws)

Found the config file in: /config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f1e14afa950>,
         subscription_id=567f42ad-44d7-4850-aada-1d69d5b9aae9,
         resource_group_name=dp-100rg,
         workspace_name=projectomegadev)


In [4]:
# get the data for AutoML job 
export_data = Input(type=AssetTypes.MLTABLE, path="azureml:cleaned-crop-export:1")

In [6]:
# set AutoML Job 
export_value_job = automl.regression(
    compute="compute-cluster",
    experiment_name="automl-crop-export-value",
    training_data=export_data,
    target_column_name="Export_Value",
    primary_metric="r2_score",
    enable_model_explainability=True,
    n_cross_validations=5
)

# set limit 
export_value_job.set_limits(
    timeout_minutes=60,
    trial_timeout_minutes=20,
    max_trials=5,
    enable_early_termination=True
)

# set automl training properties
export_value_job.set_training(
    blocked_training_algorithms=['DecisionTree'],
    enable_onnx_compatible_models=True
)

# set job featurization 
export_value_job.set_featurization(
    mode="auto"
)

# execute job 
executed_job = ws.jobs.create_or_update(export_value_job)
print(f"Job studio url: \n {executed_job.studio_url}")

Job studio url: 
 https://ml.azure.com/runs/funny_endive_cbrsmjcrlg?wsid=/subscriptions/567f42ad-44d7-4850-aada-1d69d5b9aae9/resourcegroups/dp-100rg/workspaces/projectomegadev&tid=a684a124-688f-4ad4-a587-1b2c2eb6febf
